# Data science project for the 13/04/2019 event
## purpose of the project

Le but de ce projet est de scraper les donnees pour faire une presentation


In [1]:
# importing modules for our work
#importing numpy
import numpy as np
# importing matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
# importing pandas
import pandas as pd
# importing request
import requests
import os, os.path
# importing beautifull soup
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import re
#importing the time module
import time

# Things to fix in other to have  a good scrapper

After examining the data scrapped it turned out that there are some data scrapped that the scrapper did not figure it out well. Here are the list of behaviour to fix.
1. number-of-items-sold: the scrapper seemed to performing well here but towards the end we don't have the value meanwhile it was supposed to be 0 if there was nothing so we need to figure out why its not working. I think since the scrapper is taking too long(more than 30 minutes) we can just scrapped the last page in other to tery to understand why
2. same issue with rom and ram but this can be solved by understand and solving the dual appearance of the of the value scrapped
3. fix the camera scrapper too

In [2]:
# url that will be scrapped
url = 'https://www.jumia.cm/smartphones/'
#pages_to_crawl = 20
#number of pages that will be scrapped
# here we are scrapping 25 pages which represent the number of pages under smartphones
pages_to_crawl = 25
r= requests.get(url)
#soup = BeautifulSoup(r.text, 'html.parser')
''' the for look bellow help get the link of each item and we can open each to investigate more in porder to get insight
from the data store in it'''
#links = soup.find_all(class_="link")
# getting all the links and putting them in an array
links =[]

# difining a dictionarry of data that need to be scrapped

data ={'name':[],
      'seller':[],
      'price':[],
      'number-of-items-sold': [],
      'ram': [],
      'rom': [],
      'screen-size': [],
       'color': [],
       'front-camera':[],
       'rear-camera':[],
       'number_of_sims':[]
      }

# scarping the pages
# here we define a for loop that will loop throught all the pages and save the link into an array
for p in range(1, pages_to_crawl+1):
    print('Scraping page:', p)
    r = requests.get(url, params={'page' : p})
    soup = BeautifulSoup(r.text, 'html.parser')
    # we pause our code for a second so that we should not be flaged as spam
   # time.sleep(1)
    for found  in soup.findAll('a', attrs={'class':'link', 'href': re.compile("^https://")}):
        links.append(found.get('href'))
##print (links)
#next after saving the link to each item, we open each of the and perform the data extraction on them
for i in range(0, len(links)):
    # open the link using the get method
    item_link = requests.get(links[i])
    item_link_soup =BeautifulSoup(item_link.text, 'html.parser')
    
    ''' make sure that the seller name exist since this is the brand of the phone , make sure that the brand name is not empty and also that the brand name is different from rock since we have realize that rock, universal is not a phone brand
    and also the type generic is the type of every other item not phones'''
    if item_link_soup.find('div', {'class':'sub-title'}).find('a') is not None and item_link_soup.find('div', {'class':'sub-title'}).find('a').get_text(strip=True).lower() !='generic' and (item_link_soup.find('div', {'class':'sub-title'}).find('a').get_text(strip=True).lower() !='universal') and (item_link_soup.find('div', {'class':'sub-title'}).find('a').get_text(strip=True).lower() !='rock'):
        #getting the seller name after inpection of the link we realize that we could get it from the a tag
        vendeur = item_link_soup.find('div', {'class':'sub-title'}).find('a').get_text(strip=True)
        data['seller'].append(vendeur)
        name = item_link_soup.find('h1', {'class': 'title' }).get_text(strip=True)
        #we realize that in the name of the product, the number of sims cards is indicated but not hence we will check if it specy it if not we put it at 1
        if('dual sim' in name.lower() or 'double puce' in name.lower()):
            data['number_of_sims'].append(2)
        else:
            data['number_of_sims'].append(1)
        data['name'].append(name)
        #getting the price of items
        prise = item_link_soup.find('span', {'dir':'ltr'})
        prize = prise.get_text(strip=True).replace(',', '')
        data['price'].append(float(prize))
        
        # getting the numbers of items sold
        sold = item_link_soup.find('div', {'class':'-ptxxs'})
        if sold  is not None :
            if sold.find('span', {'class': 'text color-default bold -prxs -inline-block -bold'})is not None:
                vendu = sold.find('span', {'class': 'text color-default bold -prxs -inline-block -bold'}).get_text(strip=True)
                
            
            # on the website the number of items sold are presents in form of X+ or <x so we add or substract one to reflect it
                if vendu.endswith('+'):
                    vendu1 = vendu.replace('+', '')
                    vendu2 = vendu1.replace(',', '')
                    vendu_final = float(vendu2) + 1
                    data['number-of-items-sold'].append(vendu_final)
                # here due to the fact that jumia.cm doesn't give the exact number sold, we add 1 if its said to be more than or sousract if to be less than.
                elif vendu.startswith('<'):
                    vendu1 = vendu.replace('<', '')
                    vendu2 = vendu1.replace(',', '')
                    vendu_final = float(vendu2) - 1
                    data['number-of-items-sold'].append(vendu_final)
                else :
                    data['number-of-items-sold'].append(0)
        # this else can be removed was having an error in my notebook and thought it was the problem but actually no
        else:
            ok = 'not know'
            
        #getting the color of the phone and various other 
        
        for caracteristiques in item_link_soup.find_all('div', {'class':'osh-row '}):
            value = caracteristiques.find_all('div', {'class': 'osh-col'})
            #print(value)
            for value1 in value:
            #print(value1)
                if 'couleur principale' in value[0].get_text(strip=True).lower():
                    couleur = value[1].get_text(strip=True).lower()
                    #print(couleur)
                    data['color'].append(value[1].get_text(strip=True).lower())
                    #print(value)
                elif 'taille de' in value[0].get_text(strip=True).lower():
                     data['screen-size'].append(value[1].get_text(strip=True).lower())
                elif 'ram syst' in value[0].get_text(strip=True).lower():
                    ram2 = float(value[1].get_text(strip=True).lower())
                    # if the value of the ram is 512 it means that its 512mb and we convert it accordingly
                    if ram2 ==512:
                        data['ram'].append(0.512)
                    else:
                        data['ram'].append(float(value[1].get_text(strip=True).lower()))
                elif 'capacit' in value[0].get_text(strip=True).lower():
                     data['rom'].append(float(value[1].get_text(strip=True).lower()))
                else:
                    a = 'ok'
        '''getting the camera info the site does not specify it in a simple way that we can just scrap so we are getting the
        the camera info by using the regex''' 
        if(len(item_link_soup.find_all('ul')[4].find_all('li'))>2):
            cam1 = str(item_link_soup.find_all('ul')[4].find_all('li')[2].get_text(strip= True)).lower()
            if('caméra' in cam1 or 'photo' in cam1):
                arr1 = [float(s) for s in re.findall(r'-?\d+\.?\d*', cam1)]
                arr2 = [x for x in arr1 if x != 0.0]
                if len(arr2) == 2:
                    data['rear-camera'].append(arr2[0])
                    data['front-camera'].append(arr2[1])
                elif len(arr2)==1 :
                    data['rear-camera'].append(arr2[0])
                    data['front-camera'].append(0)
                else:
                    arr3 = 0
        #data['vendeur'].append('unknow')
        # we are using sleep in other for our scrapper to waith for one second before going again
        #time.sleep(1)
        
''' We are having thinks twice so that s why we are dividing by two.'''
        
# casting colors in other to get the real color
couleur_to =list( data['color'])
couleur_ok = []
ram_ok = []
rom_ok = []
taille_ok = []
#print(couleur_to)
ram_to = list(data['ram'])
print(ram_to)
rom_to = list(data['rom'])
taille_to = list(data['screen-size'])
print(int(len(couleur_to)/2))
for i in range(int(len(taille_to)/2)):
    taille_ok.append(taille_to[i*2])
    data['screen-size'] = taille_ok
for i in range(int(len(ram_to)/2)):
    ram_ok.append(ram_to[i*2])
    data['ram'] = ram_ok
for i in range(int(len(rom_to)/2)):
    rom_ok.append(rom_to[i*2])
    data['rom'] = rom_ok
for i in range(int(len(couleur_to)/2)):
    
    #del couleur_to[i*2]
    couleur_ok.append(couleur_to[i*2])
    #ram_ok.append(ram_to[i*2])
    #rom_ok.append(rom_to[i*2])
    #taille_ok.append(taille_to[i*2])
    data['color'] = couleur_ok

#print(data)

#converting the dictionnary to a dataframe
frame = pd.concat([pd.Series(v, name=k) for k, v in data.items()], axis=1)
#print(frame)
# here we export the data to csv file
export_csv = frame.to_csv (r'./dataframe.csv', index = False)

Scraping page: 1
Scraping page: 2
Scraping page: 3
Scraping page: 4
Scraping page: 5
Scraping page: 6
Scraping page: 7
Scraping page: 8
Scraping page: 9
Scraping page: 10
Scraping page: 11
Scraping page: 12
Scraping page: 13
Scraping page: 14
Scraping page: 15
Scraping page: 16
Scraping page: 17
Scraping page: 18
Scraping page: 19
Scraping page: 20
Scraping page: 21
Scraping page: 22
Scraping page: 23
Scraping page: 24
Scraping page: 25
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 3.0, 3.0, 3.0, 1.0, 1.0, 4.0, 4.0, 4.0, 4.0, 1.0, 1.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 2.0, 2.0, 4.0, 4.0, 4.0, 4.0, 8.0, 8.0, 4.0, 4.0, 4.0, 4.0, 1.0, 1.0, 3.0, 3.0, 2.0, 2.0, 3.0, 3.0, 4.0, 4.0, 3.0, 3.0, 1.0, 1.0, 6.0, 6.0, 3.0, 3.0, 3.0, 3.0, 6.0, 6.0, 3.0, 3.0, 1.0, 1.0, 1.0, 1.0, 3.0, 3.0, 8.0, 8.0, 3.0, 3.0, 6.0, 6.0, 1.0, 1.0, 3.0, 3.0, 2.0, 2.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 2.0, 2.0, 3.0, 3.0, 3.0, 3.0

# analysing the csv file save


In [3]:
# reading the saved file
df = pd.read_csv('dataframe.csv')
print(df.head())
df.describe()

                                                name   seller    price  \
0  Spark Dual SIM - 8Go HDD - 1Go RAM - Noir - 12...   Xtouch  32100.0   
1              iPhone 4S - 16Go HDD - Noir - 12 Mois    Apple  47900.0   
2  Hot 7 Dual Sim - 6,2" - 16 Go HDD - 1 Go RAM - Or  Infinix  61500.0   
3  Rise 31 Dual SIM 8Go HDD - Bleu + 1 Carte SIM ...   Orange  24900.0   
4  Hot 7 Dual SIM - 6,2" - 16 Go HDD - 1 Go RAM -...  Infinix  61500.0   

   number-of-items-sold  ram   rom  screen-size color  front-camera  \
0                  99.0  1.0   8.0          5.0  noir           2.0   
1               10001.0  1.0  16.0          3.5  noir           0.0   
2                1001.0  1.0  16.0          6.2    or           8.0   
3                1001.0  1.0   8.0          4.0  bleu           8.0   
4                1001.0  1.0  16.0          6.2  noir           8.0   

   rear-camera  number_of_sims  
0          5.0               2  
1          8.0               1  
2         13.0               

,price,number-of-items-sold,ram,rom,screen-size,front-camera,rear-camera,number_of_sims
count,8.990000e+02,853.000000,786.000000,816.000000,769.000000,604.000000,604.000000,899.000000
mean,1.806228e+05,2020.376319,2.873547,50.071078,5.509493,6.214404,11.502649,1.515017
std,1.836937e+05,9346.948276,2.363238,93.289166,0.782874,4.026143,3.897819,0.500053
min,1.215700e+04,9.000000,0.512000,4.000000,0.000000,0.000000,0.300000,1.000000
25%,7.200000e+04,99.000000,1.000000,16.000000,5.000000,5.000000,8.000000,1.000000
50%,1.300000e+05,101.000000,3.000000,32.000000,5.600000,5.000000,12.000000,2.000000
75%,2.120000e+05,1001.000000,4.000000,64.000000,6.000000,8.000000,13.000000,2.000000
max,2.210000e+06,100001.000000,32.000000,1616.000000,12.000000,24.000000,40.000000,2.000000
